In [33]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
df1 = pd.read_csv('data/house.csv')

In [395]:
df1.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [6]:
df1.shape

(2226382, 12)

In [7]:
df1.describe()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
count,2.221849e+06,2.224841e+06,1.745065e+06,1.714611e+06,1.900793e+06,2.215516e+06,2.226083e+06,1.657898e+06
mean,5.293989e+04,5.241955e+05,3.275841e+00,2.496440e+00,1.522303e+01,1.012325e+06,5.218668e+04,2.714471e+03
std,3.064275e+04,2.138893e+06,1.567274e+00,1.652573e+00,7.628238e+02,5.837635e+05,2.895408e+04,8.081635e+05
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
25%,2.386100e+04,1.650000e+05,3.000000e+00,2.000000e+00,1.500000e-01,5.063128e+05,2.961700e+04,1.300000e+03
50%,5.288400e+04,3.250000e+05,3.000000e+00,2.000000e+00,2.600000e-01,1.012766e+06,4.838200e+04,1.760000e+03
75%,7.918300e+04,5.500000e+05,4.000000e+00,3.000000e+00,9.800000e-01,1.521173e+06,7.807000e+04,2.413000e+03
max,1.101420e+05,2.147484e+09,4.730000e+02,8.300000e+02,1.000000e+05,2.001357e+06,9.999900e+04,1.040400e+09


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB


In [22]:
df1.isnull().sum()

brokered_by         4533
status                 0
price               1541
bed               481317
bath              511771
acre_lot          325589
street             10866
city                1407
state                  8
zip_code             299
house_size        568484
prev_sold_date    734297
dtype: int64

In [105]:
df2 = df1.drop(['brokered_by','status', 'prev_sold_date'], axis = 1)

In [106]:
df3 = df2.dropna(subset=['price', 'zip_code', 'state', 'house_size', 'acre_lot', 'city'], axis = 0)

In [107]:
df3.shape

(1380967, 9)

In [108]:
df3.isnull().sum()

price             0
bed           14606
bath          17681
acre_lot          0
street         4116
city              0
state             0
zip_code          0
house_size        0
dtype: int64

In [137]:
df4 = df3[df3['street'].isnull()]
df4[['street','state','city']].drop_duplicates()

,street,state,city
397,NaN,Puerto Rico,Dorado
946,NaN,Puerto Rico,Vega Alta
2106,NaN,Puerto Rico,Santa Isabel
2323,NaN,Puerto Rico,Humacao
2377,NaN,Virgin Islands,Saint Thomas
...,...,...,...
2219226,NaN,Washington,Kelso
2220484,NaN,Washington,Leavenworth
2221753,NaN,Washington,Colbert
2221936,NaN,Washington,Airway Heights


In [ ]:
def get_street (state, city):
    mode_street = df3[(df3['state'] == state) & (df3['city'] == city)]['street'].mode()
    if not mode_street.empty:
        return mode_street.iloc[0]

In [ ]:
df3[['state','city']].unique()

In [123]:
def get_street (state, city):
    mode_street = df3[(df3['state'] == state) & (df3['city'] == city)]['street'].mode()
    if not mode_street.empty:
        return mode_street.iloc[0]

In [124]:
df4 = df3.apply(lambda x: get_street(x['state'], x['city']) if pd.isnull(x['street']) else x['street'], axis=1)

KeyboardInterrupt: 

In [409]:
from sklearn.linear_model import LinearRegression

df5 = df4.dropna(subset='price', axis=0)

X = df5.drop('price', axis=1)
y = df5['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [415]:
model.score(X_test, y_test)

0.3474335390976476